In [1]:
import os
import copy
import wget
import time
import asyncio
import warnings
import logging
from astropy import time as astropytime

import numpy as np

from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
from lsst.ts.standardscripts.auxtel.latiss import LATISS

%matplotlib inline

In [2]:
attcs = ATTCS()
latiss = LATISS(attcs.domain)
# cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

await attcs.start_task
await latiss.start_task

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 10 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 30 messages


[None, None, None, None]

RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 21 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 14 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 23 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 24 messages


In [3]:
# Declare target name
target_name='HD85055'

In [4]:
c = salobj.Controller("Script", index=3)
await c.start_task

RemoteEvent(ATMCS, 0, target) falling behind; read 10 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 10 messages


In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])

In [ ]:
await attcs.slew_icrs(ra = "09 49 17.2304091481", dec = "-08 50 06.227916336", rot_pa=-90., target_name=target_name)

# Test of axis orientation on the camera.

## 1 - Take reference image with no offset. 

In [27]:
c.evt_logMessage.set_put(message="Test of axis orientation on the camera. [START]")

True

In [28]:
c.evt_logMessage.set_put(message="1 - Take reference image with no offset. [START]")

True

In [29]:
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20, num=0) #refence offset
await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)

TimeoutError: 

In [30]:
exp_time=5.0 # seconds

group_id=astropytime.Time.now().tai.isot

image_1 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_1.imageName)

AT_O_20200127_000602


In [31]:
c.evt_logMessage.set_put(message="1 - Take reference image with no offset. [END]")

True

## 2 - Apply hexapod +0.5mm offset in the x-direction and take an exposure

In [32]:
c.evt_logMessage.set_put(message="2 - Apply hexapod +0.5mm offset in the x-direction and take an exposure [START]")

True

In [33]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=+0.5, y=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [34]:
group_id=astropytime.Time.now().tai.isot

image_2 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_2.imageName)

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


AT_O_20200127_000603


In [35]:
c.evt_logMessage.set_put(message="2 - Apply hexapod +0.5mm offset in the x-direction and take an exposure [END]")

True

## 3 - Zero offset in hexapod, do 30 arcsec offset in elevation and take an exposure

In [36]:
c.evt_logMessage.set_put(message="3 - Zero offset in hexapod, do 30 arcsec offset in elevation and take an exposure [START]")

True

In [37]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(x=-0.5, y=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20+30, num=0) #refence offset
await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)

TimeoutError: 

In [38]:
group_id=astropytime.Time.now().tai.isot

image_3 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_3.imageName)

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


AT_O_20200127_000604


In [39]:
c.evt_logMessage.set_put(message="3 - Zero offset in hexapod, do 30 arcsec offset in elevation and take an exposure [END]")

True

## 4 - apply hexapod +0.5mm offset in the y-direction and take an exposure

In [40]:
c.evt_logMessage.set_put(message="4 - apply hexapod +0.5mm offset in the y-direction and take an exposure [START]")

True

In [41]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(y=+0.5, x=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20, num=0) #refence offset
await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)

In [42]:
group_id=astropytime.Time.now().tai.isot

image_4 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_4.imageName)

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


AT_O_20200127_000605


In [43]:
c.evt_logMessage.set_put(message="4 - apply hexapod +0.5mm offset in the y-direction and take an exposure [END]")

True

## 5 - Zero offset in hexapod, do 30 arcsec offset in azimuth and take an exposure

In [44]:
c.evt_logMessage.set_put(message="5 - Zero offset in hexapod, do 30 arcsec offset in azimuth and take an exposure [START]")

True

In [45]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(y=-0.5, x=0., z=0)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
attcs.atmcs.evt_allAxesInPosition.flush()
await attcs.atptg.cmd_offsetAzEl.set_start(az=60+30, el=20, num=0) #refence offset
await attcs.atmcs.evt_allAxesInPosition.next(flush=False, timeout=30.)

In [46]:
group_id=astropytime.Time.now().tai.isot

image_5 = await latiss.take_image(exptime=exp_time, shutter=True,image_type='OBJECT',
                                  group_id=group_id, filter='KPNO_406_828nm', grating='empty_1')# 'blank_bk7_wg05' KPNO_406_828nm'  'KPNO_1111_436nm' 'KPNO_373A_677nm' 'ronchi170lpmm'

print(image_5.imageName)

RemoteEvent(ATHeaderService, 0, logMessage) falling behind; read 10 messages


AT_O_20200127_000606


In [47]:
c.evt_logMessage.set_put(message="5 - Zero offset in hexapod, do 30 arcsec offset in azimuth and take an exposure [END]")

True

In [48]:
c.evt_logMessage.set_put(message="Test of axis orientation on the camera. [END]")

True

In [ ]:
await attcs.ataos.cmd_offset.set_start(x=+0.05, y=0., z=0)

In [ ]:
await attcs.shutdown()

In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18-40, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+160., -18-50, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
az=168.56 
el=-92

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([70, 35, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)




In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=60, el=20, num=0)

In [ ]:
dz = 0.8

In [ ]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(z=-dz)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)



In [ ]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_offset.set_start(z=2.*dz)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)


## 

In [ ]:
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20-90., -18, 0.], rotation_matrix(angle))
print(az,el)
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)


In [ ]:
c = salobj.Controller("Script", index=11)

In [ ]:
await c.start_task

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)



In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
offset_x = (np.random.random()-0.5)*240.
offset_y = (np.random.random()-0.5)*240.
print(offset_x, offset_y)
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20+offset_x, -18+offset_y, 0.], rotation_matrix(angle))
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=+0.05)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

In [ ]:
await attcs.enable(settings={
    'ataos': "current",
    'atmcs': "",
    'atptg': "",
    'atpneumatics': "",
    'athexapod': "auxtel_201912_crazy.yaml",
    'atdome': "test.yaml",
    'atdometrajectory': ""})

In [ ]:
rotation_matrix = lambda angle: np.array([
            [np.cos(np.radians(angle)), -np.sin(np.radians(angle)), 0.],
            [np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0.],
            [0., 0., 1.]])
azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()
angle = np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle)
el, az, _ = np.matmul([20, -18., 0.], rotation_matrix(angle))

In [ ]:
el, az

In [ ]:
await attcs.atptg.cmd_offsetAzEl.set_start(az=az, el=el, num=0)

In [ ]:
5*360/24

In [ ]:
attcs.slew_icrs?

In [ ]:
await attcs.slew_icrs

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=x_offset)
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="y", offset=y_offset)

In [ ]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)

In [ ]:
dz = 0.8

try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=5)

azel = await attcs.atmcs.tel_mount_AzEl_Encoders.aget()
nasmyth = await attcs.atmcs.tel_mount_Nasmyth_Encoders.aget()

print(f"============")
print(f"dz={dz}")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print(f"['{intra_endron.imageName}.fits'], ['{extra_endron.imageName}.fits']")
print(np.mean(azel.elevationCalculatedAngle)+np.mean(nasmyth.nasmyth1CalculatedAngle))

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")


In [ ]:
intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'

intra_fname = wget.download(intra_wget_url, out="/home/saluser/develop/cwfs/python/data")
extra_fname = wget.download(extra_wget_url, out="/home/saluser/develop/cwfs/python/data")

print(intra_fname,extra_fname)

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=0.)

In [ ]:
await attcs.ataos.cmd_resetOffset.start()

In [ ]:
await attcs.ataos.cmd_offset.set_start(z=-0.1)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
print("done")

In [ ]:
await attcs.atdome.cmd_openShutter.start()